# Load the pretrained model

Archive Images (on Server!)

In [ ]:
tar -czvf /home/alex/all_imgs_11_10_23.tar.gz

Download archive from server and extract it

In [ ]:
scp alex@foodsnapai.com:/home/alex/all_imgs_11_10_23.tar.gz allImgs.tar.gz
tar -xzvf allImgs.tar.gz -C /home/alex/allImgs_extracted/

Load annotations without Images

In [ ]:
import os

# Get the value of the environment variable
labelstudio_token = os.getenv('LABELSTUDIO_TOKEN')
annotations_base_filename = "annotations_test1"
!curl -X GET http://foodsnapai.com:8080/api/projects/1/export?exportType=CSV -H 'Authorization: Token {labelstudio_token}' --output {annotations_base_filename +".csv"} 

Convert annotations and images and preprocess them

In [21]:
%run 'convert_csv_to_mlflow_csv.py' {annotations_base_filename +".csv"} 

In [22]:
# merge classes here
# curret merge string: not:Tomato,Cucumber,Bell-Pepper,Onion,Carrot,Banana,Egg,Scallion,Lemon,Potato,Zucchini,Garlic,Apple,Pumpkin,Ginger
%run 'convert_pascal_to_googlecsv.py' --merged_csv {annotations_base_filename +"_mlflow.csv"} 

Skipping automatic class merge.
['Tomato', 'Cucumber', 'Bell-Pepper', 'Onion', 'Carrot', 'Banana', 'Egg', 'Scallion', 'Lemon', 'Potato', 'Zucchini', 'Garlic', 'Apple', 'Pumpkin', 'Ginger']
Merging classes ['Soysauce', 'Peas', 'Oil', 'Tofu', 'Rice', 'Beans', 'Strawberry', 'Chilli', 'Lentils', 'Milk', 'Avocado', 'Spinach', 'Vinegar', 'Mango', 'Yoghurt', 'Canned-Tomato', 'Corn', 'Pasta', 'Chickpeas', 'Skyr', 'Balsamico', 'Nuts', 'Mushroom', 'Cheese', 'Plantmilk', 'Cauliflower', 'Flour', 'Broccoli', 'Orange', 'Butter', 'Curd', 'Cabagge', 'Lime', 'Eggplant', 'Salad', 'Leek'] to 'other'
Train-test-validation split applied to annotations_test1_mlflow_shuffled.csv


Now rename the file paths in the file to their actual location (find & replace)

In [25]:
import csv
import os

# Constant path to prepend
constant_path = '/home/alex/allImgs_extracted_smaller/'

# CSV file path
csv_file_path = annotations_base_filename +"_mlflow_shuffled.csv"

# Open the CSV file and modify the paths in the second column
with open(csv_file_path, 'r+', newline='') as file:
    csv_reader = csv.reader(file)
    rows = list(csv_reader)

    for row in rows:
        if len(row) > 1:  # Ensure the row has at least two columns
            # Assuming the file path is in the second column (index 1)
            file_path = row[1]
            file_name = os.path.basename(file_path)  # Extract the filename from the original path
            row[1] = os.path.join(constant_path, file_name)  # Create the new path with the constant path and filename

    # Move the file pointer to the beginning
    file.seek(0)

    # Write the modified rows back to the CSV file
    csv_writer = csv.writer(file)
    csv_writer.writerows(rows)

print(f"CSV file '{csv_file_path}' modified with constant path replacing each file path in the second column.")



CSV file 'annotations_test1_mlflow_shuffled.csv' modified with constant path replacing each file path in the second column.


In [23]:
!pwd
%run 'png_to_jpeg.py' {annotations_base_filename +"_mlflow_shuffled.csv"} 
#%run 'png_to_jpeg.py' annotations1650_mlflow_shuffled_n.csv

/home/alex/tflite_model_maker_wsl2
convert png image files to jpeg imgs


0it [00:00, ?it/s]

Converted 0
Ignored 0 files because: Not a PNG file
Ignored 11599 files because: Not found


In [ ]:

%run preproc_imgs.py /home/alex/allImgs_extracted /home/alex/allImgs_extracted_smaller 800
#%run preproc_imgs.py /mnt/z/annotated_individual/images/ /home/alex/individual_tiny_extracted_smaller 800

In [26]:

%run normalize_csv.py {annotations_base_filename +"_mlflow_shuffled.csv"}  {annotations_base_filename +"_mlflow_shuffled_n.csv"} 

check the stats of the dataset splits

In [27]:
%run 'get_label_stats.py' {annotations_base_filename +"_mlflow_shuffled_n.csv"} 

There are 1709 different files in this set
There are 16 different classes in this set
Classes: Potato, other, Onion, Bell-Pepper, Scallion, Cucumber, Apple, Lemon, Egg, Tomato, Garlic, Banana, Carrot, Ginger, Zucchini, Pumpkin
Potato occurs in 219 different files
other occurs in 1468 different files
Onion occurs in 341 different files
Bell-Pepper occurs in 379 different files
Scallion occurs in 277 different files
Cucumber occurs in 428 different files
Apple occurs in 201 different files
Lemon occurs in 244 different files
Egg occurs in 254 different files
Tomato occurs in 458 different files
Garlic occurs in 213 different files
Banana occurs in 284 different files
Carrot occurs in 304 different files
Ginger occurs in 163 different files
Zucchini occurs in 208 different files
Pumpkin occurs in 119 different files

Class Distribution in Splits:

TRAIN:
Potato: 180 files (4.02%)
other: 1181 files (26.36%)
Onion: 276 files (6.16%)
Bell-Pepper: 301 files (6.72%)
Scallion: 220 files (4.91%)

Now you can start to load the model and train!

In [28]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

model_name = 'efficientdet-lite0'
epochs = 50
batch_size = 4
model_dir = 'models/'+model_name+'/'+'new_model_1700'+'_e'+str(epochs)+'_b'+str(batch_size)
#spec = model_spec.get('efficientdet_lite1')
spec = object_detector.EfficientDetLite0Spec(
    model_name = model_name,
    model_dir='/home/alex/checkpoints_lite0_1700imgs/',
    hparams='grad_checkpoint=true,strategy=gpus',
    epochs=epochs, batch_size=batch_size,
    steps_per_execution=1, moving_average_decay=0,
    var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
    tflite_max_detections=25
)


# Load the dataset

In [29]:
import os
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)
file_path = '/home/alex/tflite_model_maker_wsl2/'+annotations_base_filename +"_mlflow_shuffled_n.csv"

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(file_path)


Current Working Directory: /home/alex/tflite_model_maker_wsl2
The file '/home/alex/tflite_model_maker_wsl2/annotations_test1_mlflow_shuffled_n.csv' exists.


# Train the model

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, validation_data=validation_data)

### Write the model maker class list to a file!

In [ ]:
import json

label_map = model.model_spec.config.label_map.as_dict()
# Writing the dictionary to a JSON file
with open(model_dir+'/label_map.json', 'w') as file:
    json.dump(label_map, file)

# run tensorboard to monitor

In [26]:
!tensorboard --logdir=/home/alex/checkpoints_lite1_1700imgs/ --port=6006

2023-11-19 00:27:54.036765: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-19 00:27:54.073041: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-19 00:27:54.073123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

E1119 00:27:54.866138 140349614286656 program.py:298] TensorBoard could not bind to port 6006, it was already in use
ERROR: TensorB

Run on test data

In [31]:
model.evaluate(test_data)

43/43 [==============================] - 7s 125ms/step



{'AP': 0.1256359,
 'AP50': 0.18864998,
 'AP75': 0.14579509,
 'APs': 0.0,
 'APm': 0.0941844,
 'APl': 0.14089024,
 'ARmax1': 0.115081616,
 'ARmax10': 0.2126694,
 'ARmax100': 0.21943912,
 'ARs': 0.0,
 'ARm': 0.1737192,
 'ARl': 0.24223559,
 'AP_/Potato': 0.08731195,
 'AP_/other': 0.23187979,
 'AP_/Onion': 0.15317023,
 'AP_/Bell-Pepper': 0.2184473,
 'AP_/Scallion': 0.11563398,
 'AP_/Cucumber': 0.16958745,
 'AP_/Apple': 0.0038759527,
 'AP_/Lemon': 0.0698714,
 'AP_/Egg': 0.36860254,
 'AP_/Tomato': 0.15523179,
 'AP_/Garlic': 0.010900743,
 'AP_/Banana': 0.29010698,
 'AP_/Carrot': 0.046379015,
 'AP_/Ginger': 6.0006e-05,
 'AP_/Zucchini': 0.013992164,
 'AP_/Pumpkin': 0.07512333}

### Export the model to tflite model

In [32]:
model.export(export_dir=model_dir)

2023-11-19 19:45:19.106179: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2023-11-19 19:45:32.672745: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


Estimated count of arithmetic ops: 1.804 G  ops, equivalently 0.902 G  MACs


2023-11-19 19:45:36.814122: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-11-19 19:45:36.814162: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-11-19 19:45:36.814653: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpjzlam0sf
2023-11-19 19:45:36.871303: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-11-19 19:45:36.871329: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpjzlam0sf
2023-11-19 19:45:37.076689: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-11-19 19:45:38.070630: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpjzlam0sf
2023-11-19 19:45:38.511198: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1696547

Estimated count of arithmetic ops: 1.804 G  ops, equivalently 0.902 G  MACs


Run inference script for visual output

In [ ]:
# needs to be after training for some reason

%run 'png_to_jpeg.py' {annotations_base_filename +"_mlflow_shuffled_n.csv"}


%run do_inference.py --input_csv {annotations_base_filename +"_mlflow_shuffled_n.csv"}\
                       --model_url {model_dir}/model.tflite --output_dir output_inference1 --label_file {model_dir}/label_map.json